In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings

In [4]:
from tensorflow.keras.applications import InceptionV3

In [5]:
pre_model = InceptionV3(
    include_top = False,
    weights = None,
    input_shape = (224, 224, 3)
)

In [6]:
pre_model.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [7]:
for layer in pre_model.layers:
    layer.trainable = False

In [8]:
last_layer = pre_model.get_layer('mixed8')

In [9]:
x = Flatten()(last_layer.output)
#x = Dense(512, activation = 'relu')(x)
#x = Dropout(0.2)(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.1)(x)
x = Dense(1, activation = 'sigmoid')(x)

In [10]:
model = Model(pre_model.input, x)

In [11]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [12]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [13]:
train_generator = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    horizontal_flip = True, width_shift_range = 0.1,
    shear_range = 0.15,
    zoom_range=0.1,
    channel_shift_range = 10
).flow_from_directory(
    'SatInfo2/train',
    target_size = (224, 224),
    class_mode = 'binary',
    batch_size = 50
)

validation_generator = ImageDataGenerator(
    rescale = 1./255.
).flow_from_directory(
    'SatInfo2/validation',
    target_size = (224, 224),
    class_mode = 'binary',
    batch_size = 50
)

Found 6402 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [15]:
model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 20
)

Epoch 1/20
129/129 [==============================] - 282s 2s/step - loss: 0.4527 - accuracy: 0.7891 - val_loss: 0.5328 - val_accuracy: 0.7233
Epoch 2/20
129/129 [==============================] - 289s 2s/step - loss: 0.4491 - accuracy: 0.7905 - val_loss: 0.5287 - val_accuracy: 0.7600
Epoch 3/20
129/129 [==============================] - 285s 2s/step - loss: 0.4385 - accuracy: 0.7988 - val_loss: 0.5499 - val_accuracy: 0.7000
Epoch 4/20
129/129 [==============================] - 288s 2s/step - loss: 0.4286 - accuracy: 0.7994 - val_loss: 0.5082 - val_accuracy: 0.7467
Epoch 5/20
129/129 [==============================] - 287s 2s/step - loss: 0.4145 - accuracy: 0.8105 - val_loss: 0.5083 - val_accuracy: 0.7667
Epoch 6/20
129/129 [==============================] - 286s 2s/step - loss: 0.3996 - accuracy: 0.8147 - val_loss: 0.5063 - val_accuracy: 0.7567
Epoch 7/20
129/129 [==============================] - 284s 2s/step - loss: 0.4019 - accuracy: 0.8201 - val_loss: 0.5088 - val_accuracy: 0.7600